## Carregar o arquivo XML em um Pandas DataFrame

In [1]:
import datetime
import json
import pandas as pd
import xmltodict

propositions = pd.DataFrame()

for year in range(1946, datetime.datetime.now().year + 1):
    path = '../data/sources/propositions/{}.xml'.format(year)
    with open(path) as file:
        contents = file.read()
    contents = xmltodict.parse(contents)
    contents = [pd.Series(dict(x))
                for x in contents['proposicoes']['proposicao']]
    contents = pd.DataFrame(contents)
    propositions = propositions.append(contents)

In [2]:
dict_cols = [
    'apreciacao',
    'autor1',
    'orgaoNumerador',
    'regime',
    'situacao',
    'tipoProposicao',
    'ultimoDespacho',
]

def dict_to_multiple_series(row, cols):
    for col, value in row[cols].items():
        series = pd.Series(dict(value))
        series.index = ['{}_{}'.format(col, x) for x in series.index]
        row = row.append(series)
    return row.drop(cols)

propositions = propositions \
    .apply(lambda row: dict_to_multiple_series(row, cols=dict_cols), axis=1)

dict_cols = [
    'situacao_orgao',
    'situacao_principal',
]
propositions = propositions \
    .apply(lambda row: dict_to_multiple_series(row, cols=dict_cols), axis=1)

In [3]:
cols = {
    'autor1_codPartido': 'category',
    'autor1_idecadastro': 'category',
    'autor1_txtNomeAutor': 'category',
    'autor1_txtSiglaPartido': 'category',
    'autor1_txtSiglaUF': 'category',
    'orgaoNumerador_id': 'category',
    'orgaoNumerador_nome': 'category',
    'orgaoNumerador_sigla': 'category',
    'qtdAutores': 'int',
    'qtdOrgaosComEstado': 'int',
    'regime_codRegime': 'category',
    'regime_txtRegime': 'category',
    'situacao_descricao': 'category',
    'situacao_id': 'category',
    'tipoProposicao_id': 'category',
    'tipoProposicao_nome': 'category',
    'tipoProposicao_sigla': 'category',
    'situacao_orgao_codOrgaoEstado': 'category',
    'situacao_orgao_siglaOrgaoEstado': 'category',
    'situacao_principal_codProposicaoPrincipal': 'category',
    'situacao_principal_proposicaoPrincipal': 'category',
}

for col, col_type in cols.items():
    propositions[col] = propositions[col].astype(col_type)

In [4]:
datetime_cols = [
    'datApresentacao',
    'ultimoDespacho_datDespacho',
]

for col in datetime_cols:
    propositions[col] = pd.to_datetime(propositions[col], dayfirst=True)

In [5]:
import numpy as np

str_cols = [
    'apreciacao_txtApreciacao',
    'regime_txtRegime',
]

propositions.loc[:, str_cols] = propositions[str_cols].replace('.', np.nan)

In [6]:
propositions.iloc[0].to_dict()

{'ano': '1946',
 'apreciacao_id': '99',
 'apreciacao_txtApreciacao': nan,
 'autor1_codPartido': nan,
 'autor1_idecadastro': nan,
 'autor1_txtNomeAutor': 'Poder Executivo',
 'autor1_txtSiglaPartido': nan,
 'autor1_txtSiglaUF': nan,
 'datApresentacao': Timestamp('1947-01-31 17:43:00'),
 'id': '529061',
 'indGenero': 'o',
 'nome': 'PL 239/1946',
 'numero': '239',
 'orgaoNumerador_id': '180',
 'orgaoNumerador_nome': 'PLENÁRIO',
 'orgaoNumerador_sigla': 'PLEN',
 'qtdAutores': 1,
 'qtdOrgaosComEstado': 1,
 'regime_codRegime': '99',
 'regime_txtRegime': nan,
 'situacao_descricao': 'Transformado em Norma Jurídica',
 'situacao_id': '1140',
 'situacao_orgao_codOrgaoEstado': '4',
 'situacao_orgao_siglaOrgaoEstado': 'MESA',
 'situacao_principal_codProposicaoPrincipal': '0',
 'situacao_principal_proposicaoPrincipal': nan,
 'tipoProposicao_id': '139',
 'tipoProposicao_nome': 'Projeto de Lei',
 'tipoProposicao_sigla': 'PL',
 'txtEmenta': 'TRANSFERE PARA O TESOURO NACIONAL PARTE DAS EMISSÕES FEITAS PA

In [7]:
cols = {
    'ano': 'year',
    'apreciacao_id': 'assessment_id',
    'apreciacao_txtApreciacao': 'assessment',
    'autor1_codPartido': 'author_party_id',
    'autor1_idecadastro': 'author_id',
    'autor1_txtNomeAutor': 'author',
    'autor1_txtSiglaPartido': 'author_party',
    'autor1_txtSiglaUF': 'author_state',
    'datApresentacao': 'presented_at',
#     'indGenero': '???',
    'nome': 'name',
    'numero': 'number',
    'orgaoNumerador_id': 'board_id',
    'orgaoNumerador_nome': 'board_name',
    'orgaoNumerador_sigla': 'board',
    'qtdAutores': 'authors',
    'qtdOrgaosComEstado': 'agencies',
    'regime_codRegime': 'regimen_id',
    'regime_txtRegime': 'regimen',
    'situacao_descricao': 'status',
    'situacao_id': 'status_id',
    'situacao_orgao_codOrgaoEstado': 'status_agency_id',
    'situacao_orgao_siglaOrgaoEstado': 'status_agency',
    'situacao_principal_codProposicaoPrincipal': 'status_proposition_id',
    'situacao_principal_proposicaoPrincipal': 'status_proposition',
    'tipoProposicao_id': 'type_id',
    'tipoProposicao_nome': 'type_name',
    'tipoProposicao_sigla': 'type',
    'txtEmenta': 'summary',
    'txtExplicacaoEmenta': 'reasoning',
    'ultimoDespacho_datDespacho': 'updated_at',
    'ultimoDespacho_txtDespacho': 'last_update_destination',
}

cols_to_drop = [
    'assessment_id',
    'author_id',
    'author_party_id',
    'board_id',
    'board_name',
    'indGenero',
    'name',
    'regimen_id',
    'status_agency_id',
    'status_id',
    'status_proposition_id',
    'type_id',
    'type_name',
]

propositions = propositions \
    .rename(columns=cols) \
    .drop(cols_to_drop, axis=1)

In [8]:
propositions.iloc[0].to_dict()

{'agencies': 1,
 'assessment': nan,
 'author': 'Poder Executivo',
 'author_party': nan,
 'author_state': nan,
 'authors': 1,
 'board': 'PLEN',
 'id': '529061',
 'last_update_destination': None,
 'number': '239',
 'presented_at': Timestamp('1947-01-31 17:43:00'),
 'reasoning': None,
 'regimen': nan,
 'status': 'Transformado em Norma Jurídica',
 'status_agency': 'MESA',
 'status_proposition': nan,
 'summary': 'TRANSFERE PARA O TESOURO NACIONAL PARTE DAS EMISSÕES FEITAS PARA ATENDER ÀS OPERAÇÕES DA CARTEIRA DE REDESCONTO DO BANCO DO BRASIL, MEDIANTE RESGATE DE DÉBITOS DO TESOURO NACIONAL AO BANCO DO BRASIL.',
 'type': 'PL',
 'updated_at': NaT,
 'year': '1946'}

In [9]:
propositions.to_csv('../data/chamber_of_deputies_propositions.csv', index=False)